In [12]:
import tweepy
import json

In [13]:
with open("conf.json","r") as f:
    conf = dict(json.loads(f.read()))

In [14]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

db = client["SMM"]
tweets_coll = db["tweets_test"]

In [15]:
class MyStreamListener(tweepy.StreamListener):
    
    def on_data(self,raw_data):
        try:
            data = json.loads(raw_data)
            data = dict(data)
            data["_id"] = data["id"]
            tweets_coll.insert_one(data)
        
        except Exception as e:
            print(e)
            
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False
        

In [16]:
consumer_key = conf["keys"]["consumer_key"]
consumer_secret = conf["keys"]["consumer_secret"]
access_token= conf["keys"]["access_token"]
access_secret = conf["keys"]["access_secret"]

In [17]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth,wait_on_rate_limit_notify=True)

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth=api.auth,listener=myStreamListener)
myStream.filter(languages=["en"],track=['spiderman'])
myStream.disconnect()